In [1]:
'''Example script to generate text from Nietzsche's writings.

At least 20 epochs are required before the generated text
starts sounding coherent.

It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.

If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
from keras.callbacks import ModelCheckpoint, EarlyStopping, LambdaCallback
from keras.layers import LSTM, Convolution1D, MaxPooling1D
import numpy as np
import random
import sys
import codecs
import unicodedata


#path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
# https://wolnelektury.pl/media/book/txt/pan-tadeusz.txt
path = get_file('pan-tadeusz.txt', origin='https://wolnelektury.pl/media/book/txt/pan-tadeusz.txt')

#with codecs.open(path, 'wb', 'utf-8') as fh:
#text = codecs.open(path,"rb",'utf-8').read().lower().decode("utf-8").encode('iso-8859-2')
text = open(path,"rb").read().lower().decode("utf-8").encode('iso-8859-2',"ignore")
#apple.decode('iso-8859-1').encode('utf8')
#text = unicodedata.normalize('NFKD', text).encode('iso-8859-2','ignore')

print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 120
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

# print (text[389:600])
# text[389:600]
# text[:140]

Using TensorFlow backend.


corpus length: 455854
total chars: 62
nb sequences: 151912


In [2]:
a=u"zażółć gęślą jaźń".encode('iso-8859-2')
a="zażółć gęślą jaźń".decode("utf-8").encode('iso-8859-2')
a=u"panicza \u2014 spotkanie".encode('iso-8859-2',"ignore")
#a.decode("utf-8").decode('iso-8859-2') #.encode('iso-8859-2')
print (a)
a

print (text[389:600].decode("iso-8859-2").encode("utf-8"))

panicza  spotkanie
   litwo! ojczyzno moja! ty jesteś jak zdrowie:
ile cię trzeba cenić, ten tylko się dowie,
kto cię stracił. dziś piękność twą w całej ozdobie
widzę i opisuję, bo tęsknię po tobie.

    panno święta, co jasn


In [3]:
print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1



Vectorization...


In [5]:

# build the model: a single LSTM
print('Build model...')
model = Sequential()
# model.add(Convolution1D(nb_filter=1024, filter_length=3, border_mode='same', activation='relu', input_shape=(maxlen, len(chars))))
# model.add(MaxPooling1D(pool_length=2))
# model.add(Convolution1D(nb_filter=512, filter_length=3, border_mode='same', activation='relu'))
# model.add(MaxPooling1D(pool_length=2))
model.add(LSTM(256, input_shape=(maxlen, len(chars))))
model.add(Dense(100))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

print (model.summary())

optimizer = "adam" #RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

# Callbacks
early_stop_cb = EarlyStopping(monitor='val_loss', patience=20, verbose=1)

# Print the batch number at the beginning of every batch.
batch_print_cb = LambdaCallback(on_batch_begin=lambda batch, logs: print (".",end=''), 
                                on_epoch_end=lambda batch, logs: print (batch))

# Plot the loss after every epoch.
plot_loss_cb = LambdaCallback(on_epoch_end=lambda epoch, logs: 
                              print (epoch, logs))
                              #plt.plot(np.arange(epoch), logs['loss']))


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y, batch_size=2048, epochs=1,
              callbacks=[batch_print_cb, plot_loss_cb],
              verbose=0)

    start_index = random.randint(0, len(text) - maxlen - 1)

    if iteration > 20: 
        for diversity in [0.2, 0.5, 1.0, 1.2]:
            print()
            print('----- diversity:', diversity)

            generated = ''
            sentence = text[start_index: start_index + maxlen]
            generated += sentence
            print('----- Generating with seed: "' + sentence.decode("iso-8859-2").encode("utf-8") + '"')
            sys.stdout.write(generated.decode("iso-8859-2").encode("utf-8"))

            for i in range(400):
                x = np.zeros((1, maxlen, len(chars)))
                for t, char in enumerate(sentence):
                    x[0, t, char_indices[char]] = 1.

                preds = model.predict(x, verbose=0)[0]
                next_index = sample(preds, diversity)
                next_char = indices_char[next_index]

                generated += next_char
                sentence = sentence[1:] + next_char

                sys.stdout.write(next_char.decode("iso-8859-2").encode("utf-8"))
                sys.stdout.flush()
            print()


Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 256)               326656    
_________________________________________________________________
dense_3 (Dense)              (None, 100)               25700     
_________________________________________________________________
dense_4 (Dense)              (None, 62)                6262      
_________________________________________________________________
activation_2 (Activation)    (None, 62)                0         
Total params: 358,618.0
Trainable params: 358,618.0
Non-trainable params: 0.0
_________________________________________________________________
None

--------------------------------------------------
Iteration 1
...........................................................................0
0 {'loss': 3.3901127940375844}

--------------------------------------------------
Iteration 2
......

In [79]:
def print_sample():
    
    for diversity in [0.01, 0.2, 0.5, 1.0, 1.2, 2.0]:
        print()
        print('----- diversity:', diversity)
        
        start_index = random.randint(0, len(text) - maxlen - 1)
        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence.decode("iso-8859-2").encode("utf-8") + '"')
        sys.stdout.write(generated.decode("iso-8859-2").encode("utf-8"))

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char.decode("iso-8859-2").encode("utf-8"))
            sys.stdout.flush()
        print()
    



In [13]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=202,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=200,
          verbose=0)

...........................................................................200
200 {'loss': 0.76143876018352907}
...........................................................................201
201 {'loss': 0.69055093807834877}


In [14]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=252,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=250,
          verbose=0)

...........................................................................250
250 {'loss': 0.60631788761935046}
...........................................................................251
251 {'loss': 0.5767314190095908}


In [17]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=302,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=300,
          verbose=0)

...........................................................................300
300 {'loss': 0.55399958481500611}
...........................................................................301
301 {'loss': 0.5348804388618047}


In [19]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=502,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=500,
          verbose=0)

...........................................................................500
500 {'loss': 0.50250559323608324}
...........................................................................501
501 {'loss': 0.4870086620346028}


In [20]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=1004,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=1000,
          verbose=0)

...........................................................................1000
1000 {'loss': 0.48368815039281338}
...........................................................................1001
1001 {'loss': 0.46264495860564148}
...........................................................................1002
1002 {'loss': 0.45362791615053782}
...........................................................................1003
1003 {'loss': 0.46974741433229145}


In [21]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=502,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=500,
          verbose=0)

...........................................................................500
500 {'loss': 0.43435527408215302}
...........................................................................501
501 {'loss': 0.43801802460494038}


In [22]:
print_sample()


----- diversity: 0.2
----- Generating with seed: "jmentu szefem,
a podówczas żył na wsi blisko obiezierza,
polowaliśmy sobie na małego źwierza.
był pokój w wielkopolsz"
jmentu szefem,
a podówczas żył na wsi blisko obiezierza,
polowaliśmy sobie na małego źwierza.
był pokój w wielkopolszcz końcu dzieciach słonę
z nim urakiej sędzio, inn

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:35: RuntimeWarning: divide by zero encountered in log


a się wszystko chatu.
słuchać w limcie, wdzię tylko baję, romsiął słonionem.

    wojano ostatni uszczać się z niemie zielę;
chy chałał głowę, że zgiestoz swoje prawiem,
rzekł  mocioże stry, tylko dostad która
wszystko udprół na pierś trzyba wnieczera;
będzie wielkami, znowu bobąw wywarzą.
i lecz złowił się na niepliczą się z drzemu.
właś

----- diversity: 0.5
----- Generating with seed: "jmentu szefem,
a podówczas żył na wsi blisko obiezierza,
polowaliśmy sobie na małego źwierza.
był pokój w wielkopolsz"
jmentu szefem,
a podówczas żył na wsi blisko obiezierza,
polowaliśmy sobie na małego źwierza.
był pokój w wielkopolszcz kary brzyk kale rodę
zrebał ruch wieczacnie i drogie ogoni,
wskiegł na kurka wtrasz, oburdzo do domu.
mam raje z genim słuch na litwie otem,
chciałem zachmało siębie, w wilają niesposię;
paci je do ładzione do rokiem nasianiem.

    otwer! ruszy z ranistwici w zęskne gosy:
temny rzeczkł, skarbąt głępiej stojrosza na soplikie.
zajrzonom chłowych, przyiły.

           

In [23]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=1002,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=1000,
          verbose=0)

...........................................................................1000
1000 {'loss': 0.41999013417076869}
...........................................................................1001
1001 {'loss': 0.40216569373114075}


In [24]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=1002,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=1000,
          verbose=0)

...........................................................................1000
1000 {'loss': 0.3863454411016507}
...........................................................................1001
1001 {'loss': 0.37850503964762883}


In [25]:
print_sample()


----- diversity: 0.2
----- Generating with seed: "jmentu szefem,
a podówczas żył na wsi blisko obiezierza,
polowaliśmy sobie na małego źwierza.
był pokój w wielkopolsz"
jmentu szefem,
a podówczas żył na wsi blisko obiezierza,
polowaliśmy sobie na małego źwierza.
był pokój w wielkopolszcz końcu dwartach długi
sakoi na smatne głowy przed pochrylował
dyskuznica, że 

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:35: RuntimeWarning: divide by zero encountered in log


te zasze łskoczył i za młodzie.
krwaw już wielk mniejsze bezach zabić, uściną
i ogromizawem ściskał w koiplerze, skiese:
byko czum gość dłużacz, na pad szelonych,
dziędło dwołko, jego tycznie czy nawe leświa
z gości wymi, ciekaje murem, oczy w rzeca robi!

    wdrzek na bęka kołpany taką wzrasawany
z rymienice

----- diversity: 0.5
----- Generating with seed: "jmentu szefem,
a podówczas żył na wsi blisko obiezierza,
polowaliśmy sobie na małego źwierza.
był pokój w wielkopolsz"
jmentu szefem,
a podówczas żył na wsi blisko obiezierza,
polowaliśmy sobie na małego źwierza.
był pokój w wielkopolszcz końcu dwartach długi
szanował ręce z ogradki, aczy odkonerta.
bracia się i jeśli kichów, biejące pupanków
odchłorać się trży młodzie: wiera o szemo smoczy,
czy wszyskne, acztenił tadeusz przez przez dwie wzgista.
po jego tak się z corziemu, że by gniatoraz
jakowej sam wydać, kim korcum podała,
mowienia w zasakonej pobczazy włości
z tylionem, że i czelowki i czysteż błynie
i każdy stono

----- di

In [27]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=1004,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=1000,
          verbose=0)

...........................................................................1000
1000 {'loss': 0.37765166725779142}
...........................................................................1001
1001 {'loss': 0.37320540810473829}
...........................................................................1002
1002 {'loss': 0.39344523639543105}
...........................................................................1003
1003 {'loss': 0.36888861132619805}


In [28]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=1004,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=1000,
          verbose=0)

...........................................................................1000
1000 {'loss': 0.44800821258072682}
...........................................................................1001
1001 {'loss': 0.35314638738547077}
...........................................................................1002
1002 {'loss': 0.32596348881551995}
...........................................................................1003
1003 {'loss': 0.30075612250044359}


In [29]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=2004,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=2000,
          verbose=0)

...........................................................................2000
2000 {'loss': 0.28887956970248646}
...........................................................................2001
2001 {'loss': 0.29712562865289105}
...........................................................................2002
2002 {'loss': 0.28067325592273296}
...........................................................................2003
2003 {'loss': 0.2877824682513021}


In [30]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=5002,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=5000,
          verbose=0)

...........................................................................5000
5000 {'loss': 0.28710647166933556}
...........................................................................5001
5001 {'loss': 0.27948027135724546}


In [31]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=6002,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=6000,
          verbose=0)

...........................................................................6000
6000 {'loss': 0.27736639297870458}
...........................................................................6001
6001 {'loss': 0.33311429654160118}


In [32]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=2004,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=2000,
          verbose=0)

...........................................................................2000
2000 {'loss': 0.28488662640478657}
...........................................................................2001
2001 {'loss': 0.24419165085111474}
...........................................................................2002
2002 {'loss': 0.24293759093571377}
...........................................................................2003
2003 {'loss': 0.24548448705643083}


In [33]:
print_sample()


----- diversity: 0.2
----- Generating with seed: "yczny litwy i europy

    litwo! ojczyzno moja! ty jesteś jak zdrowie:
ile cię trzeba cenić, ten tylko się dowie,
kt"
yczny litwy i europy

    litwo! ojczyzno moja! ty jesteś jak zdrowie:
ile cię trzeba cenić, ten tylko się dowie,
którym za męstrzamów, ksz

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:35: RuntimeWarning: divide by zero encountered in log


yże wyrodu bezachy
nie wiedział po jedeuszkach i liłące fudneje!
już weść ni kłatasem niech zgoniadzia samieny.
w etelicy, wydawadzony jak wielkie kłapie;
zarściłomi w domunie, kawart rozkowy,
w któregł wszyscyw byłach, znapów dokoczy,
chyba się szadać z szmowę roztwioszecz sam mliczenie,
kiedy z sobą podnomieństwo, bo się wie polewę;
je hrabio, niekarków rzemorskiej

----- diversity: 0.5
----- Generating with seed: "nie wiedział ni wtenczas, ni potem.
daremnie pytaniami klucznika dręczono;
nic nie wyrzekł, prócz tych słów: pro publi"
nie wiedział ni wtenczas, ni potem.
daremnie pytaniami klucznika dręczono;
nic nie wyrzekł, prócz tych słów: pro publim go zdosy!
a któraw wojskiem! wreszci, nie kolny powieca,
trzymały go.

                         trastu, spotki, przypłósł,
czyła się, lup rzą pus, i gna dumkoczył,
wszystkie wyruszyju, pod skope talej rządków łżen,
na  ziedł rękę miałem, właśny, że dzię napielkimnie,
bo wdzieciadnioją do ława, a dżiekaju *urowi.

    prójdziej zostawiwsz

In [35]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=5002,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=5000,
          verbose=0)

...........................................................................5000
5000 {'loss': 0.22902992193692229}
...........................................................................5001
5001 {'loss': 0.24949291227138704}


In [36]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=2202,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=2200,
          verbose=0)

...........................................................................2200
2200 {'loss': 0.26087823141348782}
...........................................................................2201
2201 {'loss': 0.22886047128522632}


In [37]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=2202,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=2200,
          verbose=0)

...........................................................................2200
2200 {'loss': 0.21191023657330593}
...........................................................................2201
2201 {'loss': 0.20778028163571413}


In [38]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=2302,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=2300,
          verbose=0)

...........................................................................2300
2300 {'loss': 0.19451015033728328}
...........................................................................2301
2301 {'loss': 0.20351968357314293}


In [39]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=2202,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=2200,
          verbose=0)

...........................................................................2200
2200 {'loss': 0.21518282285861115}
...........................................................................2201
2201 {'loss': 0.23425683823171226}


In [40]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=3002,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=3000,
          verbose=0)

...........................................................................3000
3000 {'loss': 0.25624194150664331}
...........................................................................3001
3001 {'loss': 0.30426601794440383}


In [42]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=9002,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=9000,
          verbose=0)

...........................................................................9000
9000 {'loss': 0.16696930196171744}
...........................................................................9001
9001 {'loss': 0.15396276338840964}


In [43]:
print_sample()


----- diversity: 0.2
----- Generating with seed: "cie.
lubił bardzo myślistwo, już to dla zabawy,
już to że odgłos trąbki i widok obławy
przypominał mu jego lata młodo"
cie.
lubił bardzo myślistwo, już to dla zabawy,
już to że odgłos trąbki i widok obławy
przypominał mu jego lata młodo sna, 

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:35: RuntimeWarning: divide by zero encountered in log


zaciął:
kuszczoś, i nie pechy dzierać o ciaste stronyczy,
zbywałem, go przyjnegłam z pejeści niedbiał w mimie,
i wili kawi roznować, a maje z bogu.
naskie krzyknął ksiawie goni, den emnej,
gdybym odchodzika suny, urotę
z rzuciodzionymiał radazy struje, urwade,
bo nowu nie na gdyby ściałe kamiecić
powęki podosowiemo nie pocewić,
mam teliżent kimku tylkł mamkiedy zabaży,
jeźna do dos

----- diversity: 0.5
----- Generating with seed: "kiedyś mierzył do mnie z góry?
utkwiłem oczy we dwie twojej broni rury;
rozpacz jakaś, żal dziwny do ziemi mnie przybi"
kiedyś mierzył do mnie z góry?
utkwiłem oczy we dwie twojej broni rury;
rozpacz jakaś, żal dziwny do ziemi mnie przybiłą;
tymczasem z mopelanków sam przy obyjaj bewały,
i tak sędzia wszyscy; nie potel z komemna.
Że leśnie kiór w siedu, a szczęśta zołazył?
ktutymi ku z wsoku; poprzudami promana,
po co powiliłani cara na swej woźny
Że mnie ku kara umnie, idzą jedą skłanął,
zarąc na pamiętanie, i szukają biady.
ledziem na przystywyku dla wzdow

In [44]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=9002,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=9000,
          verbose=0)

...........................................................................9000
9000 {'loss': 0.13874216386053234}
...........................................................................9001
9001 {'loss': 0.13617070754611843}


In [45]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=15002,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=15000,
          verbose=0)

...........................................................................15000
15000 {'loss': 0.14291397544110868}
...........................................................................15001
15001 {'loss': 0.17276819445186142}


In [46]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=9002,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=9000,
          verbose=0)

...........................................................................9000
9000 {'loss': 0.18678879550147556}
...........................................................................9001
9001 {'loss': 0.26726923347071918}


In [47]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=15002,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=15000,
          verbose=0)

...........................................................................15000
15000 {'loss': 0.30400124051144128}
...........................................................................15001
15001 {'loss': 0.23944598503214243}


In [48]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=7202,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=7200,
          verbose=0)

...........................................................................7200
7200 {'loss': 0.19574664777109574}
...........................................................................7201
7201 {'loss': 0.1446278430990362}


In [49]:
print_sample()


----- diversity: 0.2
----- Generating with seed: " zgody nigdy nie pochwalam!
to mój system! ktoś drugi wrzasnął: nie pozwalam!
inni z kątów wtórują. nareszcie głos gru"
 zgody nigdy nie pochwalam!
to mój system! ktoś drugi wrzasnął: nie pozwalam!
inni z kątów wtórują. nareszcie głos gruby.
wi

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:35: RuntimeWarning: divide by zero encountered in log


ęci, co się ostataćku usząkać podkomory,
wywieszcał rękę na sędnich czwiwaje gronicy,
lecz lecz go poworskach ze słoszą się pisa,
chciciel mopczem wypał kamch ierzdźwa cerzę!
już miasz, się tym gniewu zabłą narzcałowała,
Że był lintos jego trzeka bez trzurach na dziady.
pien, mędzioćm jaka w sroni w gniu dankiej górzem.
dziwszy urząca podkomorzy arpanymczą.
dośćki, w epali, szukał k

----- diversity: 0.5
----- Generating with seed: "wojskiego  uwaga podkomorzego nad kometami  tajemnicza scena w pokoju sędziego  tadeusz, chcąc zręcznie wyplątać się, wp"
wojskiego  uwaga podkomorzego nad kometami  tajemnicza scena w pokoju sędziego  tadeusz, chcąc zręcznie wyplątać się, wpatrąć się i leci: Żydył się napinnie słuczyi, wyrażył się złównego dłogomowych.
tad ostat to w czterych rama, pogląda w kraje,
wszytk się powrotrokuło; swoby podłastwo;
i drugą lub wieść rękęścią była się pozowiem,
a z wobim tęgo na pasach dziadości dawsze,
który miał dałoństwokaj, że zgodzini się nicy,
mawie pod każeń 

In [50]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=7202,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=7200,
          verbose=0)

...........................................................................7200
7200 {'loss': 0.17310991772710005}
...........................................................................7201
7201 {'loss': 0.16486029940414493}


In [51]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=12202,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=12200,
          verbose=0)

...........................................................................12200
12200 {'loss': 0.18465924269199824}
...........................................................................12201
12201 {'loss': 0.19280892877984532}


In [52]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=7202,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=7200,
          verbose=0)

...........................................................................7200
7200 {'loss': 0.17292186025383161}
...........................................................................7201
7201 {'loss': 0.1487839465148767}


In [53]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=7202,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=7200,
          verbose=0)

...........................................................................7200
7200 {'loss': 0.11527297808206304}
...........................................................................7201
7201 {'loss': 0.10816395448479484}


In [54]:
print_sample()


----- diversity: 0.2
----- Generating with seed: "gęski tłuste, kumpie, skrzydliki ozoru,
wszystkie wyborne, wszystkie sposobem domowym
uwędzone w kominie dymem jałowco"
gęski tłuste, kumpie, skrzydliki ozoru,
wszystkie wyborne, wszystkie sposobem domowym
uwędzone w kominie dymem jałowcować;
ws

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:35: RuntimeWarning: divide by zero encountered in log


zystko pan! taż tak nie z dobrzy jeszczy długim litci.
podpajła wrzek  hrebką gławę, jak sobie z ramienie.

    catej szerskali!   znarź hesłuskie się kowicą,
wstądzie z halbeem skryja harłani mówką podaleki,
motołu wez sza cala już skoczył w prosi,
i drogą, i jak we grzędy, i najnie wyszejenia,
on bornie starnych zarszy hajba siedział przed rajny,
ale przerwał rzeczki się; żółbe b

----- diversity: 0.5
----- Generating with seed: "w ogrodzie
nie było jej; mignęła tylko śród okienka
jej różowa wstążeczka i biała sukienka.
widać na grzędach, jaką p"
w ogrodzie
nie było jej; mignęła tylko śród okienka
jej różowa wstążeczka i biała sukienka.
widać na grzędach, jaką pszy minim do poki.
a niech jeszcze nigdy uchu. potku tylko,
kle kweszło nagot wszyscy rzierz seról niczą!
kropliki tłuszczy raz przerzega* u wzroju,
albo jak prosze zabłę. lecz przez wyjana!
tak szlachta to kochano za tysku zalata
Że na kichek, jakby oblicat i pan stać,
i książe staworszy zawogi, kulczniko hrobo.
odeszczegi si

In [55]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=9202,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=9200,
          verbose=0)

...........................................................................9200
9200 {'loss': 0.1073829133660535}
...........................................................................9201
9201 {'loss': 0.10332982048868625}


In [56]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=12202,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=12200,
          verbose=0)

...........................................................................12200
12200 {'loss': 0.11854927821086916}
...........................................................................12201
12201 {'loss': 0.22678475721707697}


In [57]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=7202,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=7200,
          verbose=0)

...........................................................................7200
7200 {'loss': 0.29045415437989303}
...........................................................................7201
7201 {'loss': 0.24449182236352812}


In [59]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=12202,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=12200,
          verbose=0)

...........................................................................12200
12200 {'loss': 0.12452230638874871}
...........................................................................12201
12201 {'loss': 0.10626985335310486}


In [60]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=15202,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=15200,
          verbose=0)

...........................................................................15200
15200 {'loss': 0.097698621008876899}
...........................................................................15201
15201 {'loss': 0.12399564685399163}


In [61]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=12202,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=12200,
          verbose=0)

...........................................................................12200
12200 {'loss': 0.14033880718850847}
...........................................................................12201
12201 {'loss': 0.13693342696836294}


In [62]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=15202,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=15200,
          verbose=0)

...........................................................................15200
15200 {'loss': 0.17356259110220124}
...........................................................................15201
15201 {'loss': 0.18854357438495772}


In [63]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=9202,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=9200,
          verbose=0)

...........................................................................9200
9200 {'loss': 0.16372268279978397}
...........................................................................9201
9201 {'loss': 0.12513523340339791}


In [64]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=12202,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=12200,
          verbose=0)

...........................................................................12200
12200 {'loss': 0.10006687035366749}
...........................................................................12201
12201 {'loss': 0.11199676718523016}


In [65]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=20002,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=20000,
          verbose=0)

...........................................................................20000
20000 {'loss': 0.098236075218201338}
...........................................................................20001
20001 {'loss': 0.12458854429104033}


In [66]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=20002,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=20000,
          verbose=0)

...........................................................................20000
20000 {'loss': 0.14790715584101299}
...........................................................................20001
20001 {'loss': 0.16486633243772478}


In [67]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=12202,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=12200,
          verbose=0)

...........................................................................12200
12200 {'loss': 0.2112594317128681}
...........................................................................12201
12201 {'loss': 0.19971048024927524}


In [68]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=40003,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=40000,
          verbose=0)

...........................................................................40000
40000 {'loss': 0.18894750321794695}
...........................................................................40001
40001 {'loss': 0.16268675468331975}
...........................................................................40002
40002 {'loss': 0.14887698201091942}


In [69]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=45005,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=45000,
          verbose=0)

...........................................................................45000
45000 {'loss': 0.12476992810753085}
...........................................................................45001
45001 {'loss': 0.10842925641956258}
...........................................................................45002
45002 {'loss': 0.07071006884961728}
...........................................................................45003
45003 {'loss': 0.054659572500531294}
...........................................................................45004
45004 {'loss': 0.046015658698110092}


In [70]:
print_sample()


----- diversity: 0.2
----- Generating with seed: "wypuści z obławy,
długo musi pracować, nim poprawi sławy.

    hrabia mówił, że pierwszy do oszczepu godził,
i że sp"
wypuści z obławy,
długo musi pracować, nim poprawi sławy.

    hrabia mówił, że pierwszy do oszczepu godził,
i że spotyć

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:35: RuntimeWarning: divide by zero encountered in log


ma odkoćć czas rościło wszamką,
i jacki raz prociszego wypalił do dowy,
cząga jesze zamyszy, co się tężą ruszwać;
szyją, ażeby strzelbę, jak zwyk zważy spocię
a chybiego chłopanka się, rzekł  choć i nie wiedziły:
bięc go w moje ubiałem, jak strachej niczy,
który przysiągło się, pośnią. ogoneju!
iż łożej ce zegan, tadem z gdzie pan powrocał,
był co tabeniesz; zaskać się pomniei rzybił;

----- diversity: 0.5
----- Generating with seed: "u! ze łzami krzyknęli;
tadeusz krzyknął: zdrowie spółobywateli,
wolnych, równych  polaków!  wnoszę ludu zdrowie!
rzek"
u! ze łzami krzyknęli;
tadeusz krzyknął: zdrowie spółobywateli,
wolnych, równych  polaków!  wnoszę ludu zdrowie!
rzekł: dywo! goradeste! bana malertałe,
jakitwiej urmawia, prósnosnojo znowós
zasiaładziniej naszyk kwiaty i umierzy:
ale się przed pochączgł zaciłem spod szlachtami,
i cząga chłopcych wydowić dwo, tela kocinę!
i pochyli z tyłkiesych spogór i nanu się goszy!

    znadóg przetrzał szlachtów mówiąż był się cali
telko który oby ko

In [71]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=60002,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=60000,
          verbose=0)

...........................................................................60000
60000 {'loss': 0.032039812791960533}
...........................................................................60001
60001 {'loss': 0.020536978852810708}


In [73]:
print_sample()


----- diversity: 0.01
----- Generating with seed: "liku,
przedstawiają polskiego historię sejmiku,
narady, wotowanie, tryumfy i waśnie;
sam tę scenę odgadłem i państwu "
liku,
przedstawiają polskiego historię sejmiku,
narady, wotowanie, tryumfy i waśnie;
sam tę scenę odgadłem i państwu oczo

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:35: RuntimeWarning: divide by zero encountered in log


tać.
nie móż  ane w samż krada wrzasnym do dokach.

    pan wozię, no księp goni, wysząpów ledzia;
bo tu iedźwiedzo, że w tyj tyt jak z prawą wkół stryk się miół!

    sięgi świat łotrzeburze. jak jencza  nie woli
szupci, i kropi tabów rozdósł  wydachale.

    raniał cuchej robiła chrdbioła  krzyknął

    jejste rozprosów letwa, na tama znośmienie!
Łaskie któty jano załeszał od ptw

----- diversity: 0.2
----- Generating with seed: "da, fanfaron, lecz mu nie brak męstwa,
na miejsce naznaczone zapewne się stawi,
rozprawim się; a jeśli bóg pobłogosław"
da, fanfaron, lecz mu nie brak męstwa,
na miejsce naznaczone zapewne się stawi,
rozprawim się; a jeśli bóg pobłogosławy,
i chciały pan. pankiem letwie onie na dziem;
onne nie sędziego, mówięcenym biebie
potkajor jej ojna nie poszalniem lasy,
pinardzoście tak przyprania caciorat o laty,
przy kwiatku in tukiść trzyma a dzie będziore:
sam podarniać nagdy nie zakała żwołogościa!
czwicim się, czas udrzemaczy: spotą się późne,

    naszelor stopl

In [74]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=70002,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=70000,
          verbose=0)

...........................................................................70000
70000 {'loss': 0.013943481075127839}
...........................................................................70001
70001 {'loss': 0.037398575230561087}


In [75]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=120002,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=120000,
          verbose=0)

...........................................................................120000
120000 {'loss': 0.35717184733576079}
...........................................................................120001
120001 {'loss': 0.48796564641612306}


In [76]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=70010,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=70000,
          verbose=0)

...........................................................................70000
70000 {'loss': 0.33128932374236497}
...........................................................................70001
70001 {'loss': 0.21753269678181938}
...........................................................................70002
70002 {'loss': 0.15157565079086782}
...........................................................................70003
70003 {'loss': 0.10045917846403313}
...........................................................................70004
70004 {'loss': 0.070240623684567335}
...........................................................................70005
70005 {'loss': 0.056979211623771064}
...........................................................................70006
70006 {'loss': 0.041881806686939617}
...........................................................................70007
70007 {'loss': 0.023544193846129199}
............................................................

In [77]:
print_sample()


----- diversity: 0.01
----- Generating with seed: " handlu zbożowym,
na wicinnym: potrzebna jest znajomość taka
na wsi. miał także sławę dobrego polaka.

    on pierws"
 handlu zbożowym,
na wicinnym: potrzebna jest znajomość taka
na wsi. miał także sławę dobrego polaka.

    on pierwszy r

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:35: RuntimeWarning: divide by zero encountered in log


ękłym w lichajto włoska wojniej rojszy

    i sołą z dno witkieruli uderżowsta czerów,
codny zwagów, a curna ogoli wybawić;
ogna żołszy za ofić, leci z ziele słuszał,
i żybraj starują się przez poglapieństwe ropiere!

    to ponawiesnycych z parszy hracia ukoroką,
czerności, spoczyłba: czaje znajdzićić!

    zajeszkł  jedrożki miał wielkie i podgiem,
od czala znaliłyśmie, jak go poz

----- diversity: 0.2
----- Generating with seed: "czny zostać się na litwie:
musicie więc do księstwa uciekać panowie,
a mianowicie maciej, co się chrzciciel zowie,
ta"
czny zostać się na litwie:
musicie więc do księstwa uciekać panowie,
a mianowicie maciej, co się chrzciciel zowie,
tadeusz och, podeszcz ich wielnajską przestania.

    a syły zwanije, sestrym mackimi więceść 
dziwej starce mowa zów, a za czmarło;
uwadza odmasz? odwodził na podkiem?
aże, po jeśli cis pięśniu na siańcu.

    otwora wrós gniewoźnie, wznosi  rzekł po wrob:
wojskiego też wiatnie się w mam talko z ziemie.
płuszałach, wzekajce ws

In [80]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=140003,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=140000,
          verbose=0)

...........................................................................140000
140000 {'loss': 0.0088851745010612732}
...........................................................................140001
140001 {'loss': 0.0078316789667880159}
...........................................................................140002
140002 {'loss': 0.0071606229592178052}


In [81]:
print_sample()


----- diversity: 0.01
----- Generating with seed: "roku,
ustępują mu z drogi,  i zmieniwszy szyki,
puszczają się znów za nim. 

                        brzmią zewsząd "
roku,
ustępują mu z drogi,  i zmieniwszy szyki,
puszczają się znów za nim. 

                        brzmią zewsząd jaki,

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:35: RuntimeWarning: divide by zero encountered in log


 bię!
to pograć nikt woła wszyscy, że drzwi colą;
prewez zdrodzie krójce! przes, bo wyjski rabońskiech,
dziwca cięca starca; zalewsła z degodzie.
szersi ostatnij tak przed skronich ziami przecisząć,
obchać się znawożyć? co ta dzię strażył?
jast wzdłościa, co odzowi do strzele za dziecie.
nadziej żednej jaki powietra dalnejeli;
nie hrabia, który dobry dla szczerdew nieddączem:
wysada.

----- diversity: 0.2
----- Generating with seed: "scy z litwą natarli w zawody,
i pomimo dawniejsze dwóch stronnictw niezgody
walczą jak bracia, jeden drugiego zachęca."
scy z litwą natarli w zawody,
i pomimo dawniejsze dwóch stronnictw niezgody
walczą jak bracia, jeden drugiego zachęca.
dzisze, że przerwał, że ty rosnęła drugami,
jakordżajestrach na dziedkich ce dzieci;
chczas moskalać się: dzie się, umiało zaru!
promby deusy, jakby pod nim nierzianów,
jako nabietku musza z podprastwiję,
wrzek po calepsuka a dzieci nawyta?
tak, truszy: jak jak śnie koła płową,
boglidowił w radę kapęty powroty,
szymejstwe

In [82]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=280003,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=280000,
          verbose=0)

...........................................................................280000
280000 {'loss': 0.0066385299049855767}
...........................................................................280001
280001 {'loss': 0.0062225929222960979}
...........................................................................280002
280002 {'loss': 0.0059421002075991875}


In [83]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=600003,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=600000,
          verbose=0)

...........................................................................600000
600000 {'loss': 0.0056402107100488258}
...........................................................................600001
600001 {'loss': 0.0052660303177108278}
...........................................................................600002
600002 {'loss': 0.0049965311163732602}


In [84]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=600002,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=600000,
          verbose=0)

...........................................................................600000
600000 {'loss': 0.0047649254982091282}
...........................................................................600001
600001 {'loss': 0.0045431612531020727}


In [85]:
print_sample()


----- diversity: 0.01
----- Generating with seed: "cz o mniej trudnych i mądrych przedmiotach,
o wiejskiego pożycia nudach i kłopotach,
i jak bawić się trzeba, i jak cza"
cz o mniej trudnych i mądrych przedmiotach,
o wiejskiego pożycia nudach i kłopotach,
i jak bawić się trzeba, i jak czasuny ja

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:35: RuntimeWarning: divide by zero encountered in log


ka!
jameu rani: mógł; przygnąłtem budalośli, gościeny, wilki różno  rudać o sów! ja reszecku!
opa: moje zniedzićm: ale już malcowa rzeczny;
pieni bagnął, chodzał a czyli wanowanku!
wręcił promie przyszyko, nie się poszpierzążeć;
bo powraca po cocają i panną słowa,
którymi od ode pczymy krejowusł
i psówow ochodziło się od tojowycie?
i wie podrożem okwyta sztupanki ziadnie,
to wydzas

----- diversity: 0.2
----- Generating with seed: "ństwu wyznać bez ogródki,
Że z tych niewczesnych bojów złe wynikną skutki.
zbłądziliśmy i nikt tu z nas nie jest bez w"
ństwu wyznać bez ogródki,
Że z tych niewczesnych bojów złe wynikną skutki.
zbłądziliśmy i nikt tu z nas nie jest bez wodziły,
które bracie co z zapędzyszniejszę dziwać;
tu spod i nieme przez pogomeństwa i piska,
i niech panowiem zabytle na kłasnia doskienia
zafaro oddawołogo w zóżba bez skraniów;
Że hoj zddać zaprasznych zastrowski, licza nażny,
ten rękają znawiej było rozmiarsze zarady.

    pan wrzebi go wyzecie potrzyło i mali,
aż czasze

In [86]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=1200002,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=1200000,
          verbose=0)

...........................................................................1200000
1200000 {'loss': 0.0043421108348729535}
...........................................................................1200001
1200001 {'loss': 0.0041686132788551029}


In [87]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=1200002,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=1200000,
          verbose=0)

...........................................................................1200000
1200000 {'loss': 0.0039963840991392025}
...........................................................................1200001
1200001 {'loss': 0.003842428486053256}


In [88]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=1200002,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=1200000,
          verbose=0)

...........................................................................1200000
1200000 {'loss': 0.0036968121779523957}
...........................................................................1200001
1200001 {'loss': 0.0035567061331240233}


In [89]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=2000002,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=2000000,
          verbose=0)

...........................................................................2000000
2000000 {'loss': 0.0034328912584864137}
...........................................................................2000001
2000001 {'loss': 0.0033192446421542675}


In [91]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=2000006,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=2000000,
          verbose=0)

...........................................................................2000000
2000000 {'loss': 0.003205807142390678}
...........................................................................2000001
2000001 {'loss': 0.0030982196036997162}
...........................................................................2000002
2000002 {'loss': 0.0029961715038037723}
...........................................................................2000003
2000003 {'loss': 0.0029032785882740276}
...........................................................................2000004
2000004 {'loss': 0.0028159344063852934}
...........................................................................2000005
2000005 {'loss': 0.0027282078247054357}


In [92]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=2000006,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=2000000,
          verbose=0)

...........................................................................2000000
2000000 {'loss': 0.0026475613635103799}
...........................................................................2000001
2000001 {'loss': 0.0025709091507285274}
...........................................................................2000002
2000002 {'loss': 0.002496603614266102}
...........................................................................2000003
2000003 {'loss': 0.0024279000234849833}
...........................................................................2000004
2000004 {'loss': 0.0023617913235326939}
...........................................................................2000005
2000005 {'loss': 0.87819451418707195}


In [ ]:
# train the model, output generated text after each iteration
model.fit(X, y, batch_size=2048, epochs=70002,
          callbacks=[batch_print_cb, plot_loss_cb],
          initial_epoch=70000,
          verbose=0)